In [1]:
local = True
data_transform = False
# transform_method = "standardscaler"  
# transform_method = "rankgauss"

from sklearn.preprocessing import QuantileTransformer
from sklearn import datasets
from sklearn import model_selection


In [2]:
! pip install torch==1.9.0+cu111 torchvision==0.10.0+cu111 torchaudio===0.9.0 -f https://download.pytorch.org/whl/torch_stable.html
# !pip install torch==1.10

Looking in links: https://download.pytorch.org/whl/torch_stable.html


In [3]:
import torch
torch.__version__

'1.9.0+cu111'

# About this notebook
- Deberta-v3-large starter code
- pip wheels is [here](https://www.kaggle.com/code/yasufuminakama/pppm-pip-wheels)
- Training notebook is [here](https://www.kaggle.com/code/yasufuminakama/pppm-deberta-v3-large-baseline-w-w-b-train)

If this notebook is helpful, feel free to upvote :)

In [4]:
# ====================================================
# Directory settings
# ====================================================
exp_names = [
    # 'albert-base-v2',
    # "roberta-base",
    # "microsoft-mpnet-base",
    "funnel-transformer-large512",
    "deberta-v3-base",
    "microsoft-deberta-large",
    "bert-base-uncased",
    "microsoft-mpnet-base-origin",
    # "microsoft-deberta-large-simple",
    # "AI-Growth-Lab-PatentSBERTa",
    # "bert-large-uncased-whole-word-masking-finetuned-squad", 
    
    # "AI-Growth-Lab-PatentSBERTa-transformer",
    
    # "xlnet-base-cased",
             
            ]

# stacking_exp_name = f"4model_stacking_1dcnn_{transform_method}"
stacking_exp_name = f"{len(exp_names)}model_stacking-origin/mlp/"
import os
if local:
    INPUT_DIR = '../../data/us-patent-phrase-to-phrase-matching/'
    
    OUTPUT_DIR = f"./output/{stacking_exp_name}/"
else:
    INPUT_DIR = '../input/us-patent-phrase-to-phrase-matching/'
    OUTPUT_DIR = './'
if not os.path.exists(OUTPUT_DIR):
    os.makedirs(OUTPUT_DIR)

In [5]:
def get_logger(filename=OUTPUT_DIR+'train'):
    from logging import getLogger, INFO, StreamHandler, FileHandler, Formatter
    logger = getLogger(__name__)
    logger.setLevel(INFO)
    handler1 = StreamHandler()
    handler1.setFormatter(Formatter("%(message)s"))
    handler2 = FileHandler(filename=f"{filename}.log")
    handler2.setFormatter(Formatter("%(message)s"))
    logger.addHandler(handler1)
    logger.addHandler(handler2)
    return logger

LOGGER = get_logger()

# CFG

In [6]:
# ====================================================
# CFG
# ====================================================
class CFG:
    
    #######実行ごとに変更必要の可能性あり
    # tree_path = "./output/6model_stacking_lgbm_xgb/"
    EXP_NAMES = exp_names
    models=[
        # 'albert-base-v2',
        #     "roberta-base",
            # "microsoft/mpnet-base",
            "funnel-transformer-large512",
            "microsoft/deberta-v3-base",
            "microsoft/deberta-v3-large",
            "bert-base-uncased",
        "microsoft/mpnet-base",
        # 'microsoft/deberta-large',
        # "AI-Growth-Lab/PatentSBERTa",
        # "bert-large-uncased-whole-word-masking-finetuned-squad"
        

        # "AI-Growth-Lab/PatentSBERTa",
        # "xlnet-base-cased"
           ]
    folder_names = [
        # 'albert-base-v2',
        #             "roberta-base",
                   #  "mpnetbase",
                   #  "funnellarge512fold4",
                   # "debertabase",
                   # "debertalarge",
                    # "bertbase"
                  ]
    
    
    max_lens = [
        133, 133, 133,
         125, 125,
    ]
    pass_folds_flg = False
    pass_folds  = [
        -1,
        -1, -1, -1
    ]
    
    #######
    paths = []
    config_paths = []
    model_paths = []
    for EXP_NAME in EXP_NAMES:
        if local:
            path=f"../exp4/output/{EXP_NAME}/"
            config_path=path+'config.pth'
            model_path=f'../exp4/output/{EXP_NAME}/'
            cpc_path = f"{INPUT_DIR}/cpc_texts.pth"
        else:
            path=f"../input/{folder_name}/{EXP_NAME}/"
            config_path=path+'config.pth'
            model_path=f'../input/{folder_name}/'
            cpc_path = "../input/pppm-deberta-v3-large-baseline-w-w-b-train/cpc_texts.pth"
        paths.append(path)
        config_paths.append(config_path)
        model_paths.append(model_path)
        
    num_workers=4
    hidden_states = []
    for model in models:
        if "small" in model:
            hidden_state  = 512
        elif "base" in model:
            hidden_state  = 768
        elif "large" in model:
            hidden_state  = 1024
        elif "xlarge" in model:
            hidden_state =  1536
        hidden_states.append(hidden_state)
    
    batch_size=32
    fc_dropout=0.2
    target_size=1
    max_len=133
    seed=42
    n_fold=4
    trn_fold=[i for i in range(n_fold)]
    pass_fold = []
#     torch.load(CFG.model_path+f"{CFG.model.replace('/', '-')}_fold{fold}_best.pth",
    

# Library

In [7]:
# ====================================================
# Library
# ====================================================
import os
import gc
import re
import ast
import sys
import copy
import json
import time
import math
import shutil
import string
import pickle
import random
import joblib
import itertools
from pathlib import Path
import warnings
warnings.filterwarnings("ignore")

import scipy as sp
import numpy as np
import pandas as pd
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
from tqdm.auto import tqdm
from sklearn.metrics import f1_score
from sklearn.model_selection import StratifiedKFold, GroupKFold, KFold

import torch
print(f"torch.__version__: {torch.__version__}")
import torch.nn as nn
from torch.nn import Parameter
import torch.nn.functional as F
from torch.optim import Adam, SGD, AdamW
from torch.utils.data import DataLoader, Dataset

if not local:
    os.system('pip uninstall -y transformers')
    os.system('pip uninstall -y tokenizers')
    os.system('python -m pip install --no-index --find-links=../input/pppm-pip-wheels-dataset transformers')
    os.system('python -m pip install --no-index --find-links=../input/pppm-pip-wheels-dataset tokenizers')
import tokenizers
import transformers
print(f"tokenizers.__version__: {tokenizers.__version__}")
print(f"transformers.__version__: {transformers.__version__}")
from transformers import AutoTokenizer, AutoModel, AutoConfig
from transformers import get_linear_schedule_with_warmup, get_cosine_schedule_with_warmup
%env TOKENIZERS_PARALLELISM=true

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# device = torch.device("cpu")
device

torch.__version__: 1.9.0+cu111
tokenizers.__version__: 0.11.6
transformers.__version__: 4.17.0
env: TOKENIZERS_PARALLELISM=true


device(type='cuda')

In [8]:
import numpy as np
import random
import pandas as pd
from copy import deepcopy as dp

from sklearn import preprocessing
from sklearn.metrics import log_loss
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.feature_selection import VarianceThreshold

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.nn.modules.loss import _WeightedLoss


# Utils

In [9]:
# ====================================================
# Utils
# ====================================================
def get_score(y_true, y_pred):
    score = sp.stats.pearsonr(y_true.reshape(-1), y_pred.reshape(-1))[0]
    return score




def seed_everything(seed=42):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    
seed_everything(seed=42)

# OOF

In [10]:
# oof_df = pd.read_pickle(CFG.path+'oof_df.pkl')
# labels = oof_df['score'].values
# preds = oof_df['pred'].values
# score = get_score(labels, preds)
# LOGGER.info(f'CV Score: {score:<.4f}')
oof_dfs = pd.DataFrame()
for path in CFG.paths:
    oof_file = [f for f in os.listdir(path) if "oof_df.pkl" in f][0]
    oof_df = pd.read_pickle(path+oof_file)
    labels = oof_df['score'].values
    preds = oof_df[["id",'pred']]
    score = get_score(labels, preds["pred"].values)
    LOGGER.info(f'CV Score: {score:<.4f}')
    oof_dfs = pd.concat([oof_dfs, oof_df[["pred"]]],axis=1)
# oof_dfs.columns = [c+str(i) for i, c in enumerate(oof_dfs.columns)]
# oof_dfs["id"] = preds["id"?]


CV Score: 0.8485
CV Score: 0.8422
CV Score: 0.8574
CV Score: 0.8072
CV Score: 0.8159


In [11]:

# xgb_df = pd.read_csv(CFG.tree_path+"oof_xgb_df.csv")[["score"]]
# lgbm_df = pd.read_csv(CFG.tree_path+"oof_lgbm_df.csv")[["score"]]
# oof_dfs = pd.concat([oof_dfs, xgb_df, lgbm_df], axis=1)


In [12]:
oof_dfs.columns = [c+str(i) for i, c in enumerate(oof_dfs.columns)]
oof_dfs["id"] = preds["id"]
oof_dfs.head()

pred0     pred1     pred2     pred3     pred4                id
0  0.000880  0.003149  0.000215  0.003350  0.030808  54c1e3b9184cb5b6
1  0.236394  0.247720  0.241680  0.315298  0.276099  ef2d4c2e6bbb208d
2  0.428095  0.501632  0.503771  0.448882  0.491518  4c3f2750e7540ab7
3  0.482324  0.536995  0.479786  0.491129  0.506613  bfd7270f57530991
4  0.273144  0.073328  0.005082  0.483306  0.448548  cc96541d4987b399

In [13]:
# oof_dfs[oof_dfs.columns[:-1]].mean(axis=1).sum()

# Data Loading

In [14]:
# # ====================================================
# # Data Loading
# # ====================================================
# test = pd.read_csv(INPUT_DIR+'test.csv')
# submission = pd.read_csv(INPUT_DIR+'sample_submission.csv')
# print(f"test.shape: {test.shape}")
# print(f"submission.shape: {submission.shape}")
# display(test.head())
# display(submission.head())

In [15]:
# # ====================================================
# # CPC Data
# # ====================================================
# cpc_texts = torch.load(CFG.cpc_path)
# test['context_text'] = test['context'].map(cpc_texts)
# display(test.head())

In [16]:
# test['text'] = test['anchor'] + '[SEP]' + test['target'] + '[SEP]'  + test['context_text']
# display(test.head())

# tokenizer

In [17]:
# # ====================================================
# # tokenizer
# # ====================================================
# # CFG.tokenizer = AutoTokenizer.from_pretrained(CFG.path+'tokenizer/')
# # CFG.tokenizer = AutoTokenizer.from_pretrained("../input/roberta-large-tokenizer/tokenizer/", use_fast=False)

# CFG.tokenizer = AutoTokenizer.from_pretrained(CFG.path+"tokenizer/",use_fast=False)
# # CFG.path+'tokenizer/'
# # AutoTokenizer.from_pretrained('../input/robertalarge/roberta-large/2022-04-23-20-31-47tokenizer')

In [18]:
# AutoTokenizer.from_pretrained('../input/robertalarge/roberta-large/2022-04-23-20-31-47tokenizer', use_fast=True)

# Dataset

In [19]:
# # ====================================================
# # Dataset
# # ====================================================
# def prepare_input(cfg, text):
#     inputs = cfg.tokenizer(text,
#                            add_special_tokens=True,
#                            max_length=cfg.max_len,
#                            padding="max_length",
#                            return_offsets_mapping=False)
#     for k, v in inputs.items():
#         inputs[k] = torch.tensor(v, dtype=torch.long)
#     return inputs


# class TestDataset(Dataset):
#     def __init__(self, cfg, df):
#         self.cfg = cfg
#         self.texts = df['text'].values

#     def __len__(self):
#         return len(self.texts)

#     def __getitem__(self, item):
#         inputs = prepare_input(self.cfg, self.texts[item])
#         return inputs

# Model

In [20]:
# # ====================================================
# # Model
# # ====================================================
# class CustomModel(nn.Module):
#     def __init__(self, cfg, config_path=None, pretrained=False):
#         super().__init__()
#         self.cfg = cfg
#         if config_path is None:
#             self.config = AutoConfig.from_pretrained(cfg.model, output_hidden_states=True)
#         else:
#             self.config = torch.load(config_path)
#         if pretrained:
#             self.model = AutoModel.from_pretrained(cfg.model, config=self.config)
#         else:
#             self.model = AutoModel.from_config(self.config)
#         self.fc_dropout = nn.Dropout(cfg.fc_dropout)
#         self.fc = nn.Linear(self.config.hidden_size, self.cfg.target_size)
#         self._init_weights(self.fc)
#         self.attention = nn.Sequential(
#             nn.Linear(self.config.hidden_size, cfg.hidden_state),
#             nn.Tanh(),
#             nn.Linear(cfg.hidden_state, 1),
#             nn.Softmax(dim=1)
#         )
#         self._init_weights(self.attention)
        
#     def _init_weights(self, module):
#         if isinstance(module, nn.Linear):
#             module.weight.data.normal_(mean=0.0, std=self.config.initializer_range)
#             if module.bias is not None:
#                 module.bias.data.zero_()
#         elif isinstance(module, nn.Embedding):
#             module.weight.data.normal_(mean=0.0, std=self.config.initializer_range)
#             if module.padding_idx is not None:
#                 module.weight.data[module.padding_idx].zero_()
#         elif isinstance(module, nn.LayerNorm):
#             module.bias.data.zero_()
#             module.weight.data.fill_(1.0)
        
#     def feature(self, inputs):
#         outputs = self.model(**inputs)
#         last_hidden_states = outputs[0]
#         # feature = torch.mean(last_hidden_states, 1)
#         weights = self.attention(last_hidden_states)
#         feature = torch.sum(weights * last_hidden_states, dim=1)
#         return feature

#     def forward(self, inputs):
#         feature = self.feature(inputs)
#         output = self.fc(self.fc_dropout(feature))
#         return output

# inference

In [21]:
# # ====================================================
# # inference
# # ====================================================
# def inference_fn(test_loader, model, device):
#     preds = []
#     model.eval()
#     model.to(device)
#     tk0 = tqdm(test_loader, total=len(test_loader))
#     for inputs in tk0:
#         for k, v in inputs.items():
#             inputs[k] = v.to(device)
#         with torch.no_grad():
#             y_preds = model(inputs)
#         preds.append(y_preds.sigmoid().to('cpu').numpy())
#     predictions = np.concatenate(preds)
#     return predictions

In [22]:
# all_predictions = pd.DataFrame()
# for exp_num in range(len(CFG.EXP_NAMES)):
#     test_dataset = TestDataset(CFG, test)
#     test_loader = DataLoader(test_dataset,
#                              batch_size=CFG.batch_size,
#                              shuffle=False,
#                              num_workers=CFG.num_workers, pin_memory=True, drop_last=False)
#     predictions = []
#     for fold in CFG.trn_fold:
#         if fold in CFG.pass_fold:
#             continue
#         model = CustomModel(CFG, config_path=CFG.config_paths[exp_num], pretrained=False)
#         if local:
#             state = torch.load(CFG.model_paths[exp_num]+f"{CFG.models[exp_num].replace('/', '-')}_fold{fold}_best.pth",
#                            map_location=torch.device('cpu'))
#         else:
#             state = torch.load(CFG.model_paths[exp_num]+f"{CFG.EXP_NAMES[exp_name]}/{CFG.models[exp_name].replace('/', '-')}_fold{fold}_best.pth",
#                            map_location=torch.device('cpu'))
#         model.load_state_dict(state['model'])
#         prediction = inference_fn(test_loader, model, device)
#         predictions.append(prediction)
#         del model, state, prediction; gc.collect()
#         torch.cuda.empty_cache()
#     predictions = np.mean(predictions, axis=0)
#     all_predictions = pd.concat([all_predictions,pd.DataFrame(predictions)], axis=1)

# # pd.concat([pd.DataFrame(predictions),pd.DataFrame(predictions)], axis=1)

# 1DCNN data

In [23]:
class oneDCNN(nn.Module):
#https://www.kaggle.com/code/lililycai/1dcnn-pytorch-moa
        def __init__(self, num_features, num_targets, hidden_size, cfg=None):
            self. model_name="1dcnn"
            super(oneDCNN, self).__init__()
            hidden_size = 1024
            cha_1 = 256
            cha_2 = 512
            cha_3 = 512

            cha_1_reshape = int(hidden_size/cha_1)
            cha_po_1 = int(hidden_size/cha_1/2)
            cha_po_2 = int(hidden_size/cha_1/2/2) * cha_3

            self.cha_1 = cha_1
            self.cha_2 = cha_2
            self.cha_3 = cha_3
            self.cha_1_reshape = cha_1_reshape
            self.cha_po_1 = cha_po_1
            self.cha_po_2 = cha_po_2

            self.batch_norm1 = nn.BatchNorm1d(num_features)
            self.dropout1 = nn.Dropout(0.1)
            self.dense1 = nn.utils.weight_norm(nn.Linear(num_features, hidden_size))

            self.batch_norm_c1 = nn.BatchNorm1d(cha_1)
            self.dropout_c1 = nn.Dropout(0.1)
            self.conv1 = nn.utils.weight_norm(nn.Conv1d(cha_1,cha_2, kernel_size = 5, stride = 1, padding=2,  bias=False),dim=None)

            self.ave_po_c1 = nn.AdaptiveAvgPool1d(output_size = cha_po_1)

            self.batch_norm_c2 = nn.BatchNorm1d(cha_2)
            self.dropout_c2 = nn.Dropout(0.1)
            self.conv2 = nn.utils.weight_norm(nn.Conv1d(cha_2,cha_2, kernel_size = 3, stride = 1, padding=1, bias=True),dim=None)

            self.batch_norm_c2_1 = nn.BatchNorm1d(cha_2)
            self.dropout_c2_1 = nn.Dropout(0.3)
            self.conv2_1 = nn.utils.weight_norm(nn.Conv1d(cha_2,cha_2, kernel_size = 3, stride = 1, padding=1, bias=True),dim=None)

            self.batch_norm_c2_2 = nn.BatchNorm1d(cha_2)
            self.dropout_c2_2 = nn.Dropout(0.2)
            self.conv2_2 = nn.utils.weight_norm(nn.Conv1d(cha_2,cha_3, kernel_size = 5, stride = 1, padding=2, bias=True),dim=None)

            self.max_po_c2 = nn.MaxPool1d(kernel_size=4, stride=2, padding=1)

            self.flt = nn.Flatten()

            self.batch_norm3 = nn.BatchNorm1d(cha_po_2)
            self.dropout3 = nn.Dropout(0.2)
            self.dense3 = nn.utils.weight_norm(nn.Linear(cha_po_2, num_targets))

        def forward(self, x):

            x = self.batch_norm1(x)
            x = self.dropout1(x)
            x = F.celu(self.dense1(x), alpha=0.06)

            x = x.reshape(x.shape[0],self.cha_1,
                          self.cha_1_reshape)

            x = self.batch_norm_c1(x)
            x = self.dropout_c1(x)
            x = F.relu(self.conv1(x))

            x = self.ave_po_c1(x)

            x = self.batch_norm_c2(x)
            x = self.dropout_c2(x)
            x = F.relu(self.conv2(x))
            x_s = x

            x = self.batch_norm_c2_1(x)
            x = self.dropout_c2_1(x)
            x = F.relu(self.conv2_1(x))

            x = self.batch_norm_c2_2(x)
            x = self.dropout_c2_2(x)
            x = F.relu(self.conv2_2(x))
            x =  x * x_s

            x = self.max_po_c2(x)

            x = self.flt(x)

            x = self.batch_norm3(x)
            x = self.dropout3(x)
            x = self.dense3(x)

            return x

   

In [24]:
class StackingDataset(Dataset):
    def __init__(self, df, features, labels=None, model_name=None):
        self.df = df[features].values
        if model_name == "2dcnn":
            # [N, Models, Labels, Channel] -> [N, Channel, Models, Labels]
            self.df = self.df.reshape(-1, len(features), 1, 1)
            self.df = self.df.transpose(0,3,1,2)

        self.labels = labels.values
        
    def __len__(self, ):
        return len(self.df)
    
    
    def __getitem__(self, item):
        inputs =  torch.FloatTensor(self.df[item]).float()
        
        if self.labels is None:
            return inputs
        labels = torch.tensor(self.labels[item]).float()
        return inputs, labels
    

In [25]:
# class twoDCNN(nn.Module):
#     def __init__(self, num_features, num_targets, hidden_size):
#         self.model_name = "2dcnn"
#         self.num_features = num_features
#         self.num_targets = num_targets
#         super(twoDCNN, self).__init__()
#         self.conv1 = nn.Conv2d(in_channels=1, out_channels=8, kernel_size=(3, 1), bias=False)
#         self.relu1 = nn.ReLU()
#         self.conv2 = nn.Conv2d(in_channels=8, out_channels=16, kernel_size=(2, 1), bias=False)
#         self.relu2 = nn.ReLU()
        
#         self.conv3 = nn.Conv2d(in_channels=16, out_channels=32, kernel_size=(2, 1), bias=False)
#         self.relu3 = nn.ReLU()
#         self.conv4 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=(3, 1), bias=False)
#         self.relu4 = nn.ReLU()
        
#         self.conv5 = nn.Conv2d(in_channels=64, out_channels=128, kernel_size=(2, 1), bias=False)
#         self.relu5 = nn.ReLU()
#         self.conv6 = nn.Conv2d(in_channels=128, out_channels=1024, kernel_size=(1, 1), bias=False)
#         self.relu6 = nn.ReLU()
            
#         self.flatten = nn.Flatten()
#         self.fc1 = nn.Linear(in_features=1024, out_features=512)
#         self.relu3 = nn.ReLU()
#         self.fc2 = nn.Linear(in_features=512, out_features=num_targets)
        

#     def forward(self, x):
#         x = self.conv1(x)
#         x = self.relu1(x)
#         x = self.conv2(x)
#         x = self.relu2(x)
        
#         x = self.conv3(x)
#         x = self.relu3(x)
#         x = self.conv4(x)
#         x = self.relu4(x)
        
#         x = self.conv5(x)
#         x = self.relu5(x)
#         x = self.conv6(x)
#         x = self.relu6(x)

#         x = self.flatten(x)
#         x = self.fc1(x)
#         x = self.relu3(x)
#         x = self.fc2(x)
#         return x


In [26]:
# class twoDCNN(nn.Module):
#     def __init__(self, num_features, num_targets, hidden_size, cfg):
#         self.model_name = "2dcnn"
#         self.num_features = num_features
#         self.num_targets = num_targets
#         super(twoDCNN, self).__init__()
#         self.conv1 = nn.Conv2d(in_channels=1, out_channels=8, kernel_size=(3, 1), bias=False)
#         self.relu1 = nn.ReLU()
#         self.conv2 = nn.Conv2d(in_channels=8, out_channels=16, kernel_size=(2, 1), bias=False)
#         self.relu2 = nn.ReLU()
        
#         # self.conv3 = nn.Conv2d(in_channels=16, out_channels=32, kernel_size=(2, 1), bias=False)
#         # self.relu3 = nn.ReLU()
#         # self.conv4 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=(3, 1), bias=False)
#         # self.relu4 = nn.ReLU()
        
# #         self.conv5 = nn.Conv2d(in_channels=64, out_channels=128, kernel_size=(2, 1), bias=False)
# #         self.relu5 = nn.ReLU()
# #         self.conv6 = nn.Conv2d(in_channels=128, out_channels=1024, kernel_size=(1, 1), bias=False)
# #         self.relu6 = nn.ReLU()
            
#         self.flatten = nn.Flatten()
#         self.fc1 = nn.Linear(in_features=80, out_features=512)
#         # self.fc1 = nn.Linear(in_features=80, out_features=512)
#         self.relu3 = nn.ReLU()
#         self.fc2 = nn.Linear(in_features=512, out_features=num_targets)
        

#     def forward(self, x):
#         x = self.conv1(x)
#         x = self.relu1(x)
#         x = self.conv2(x)
#         x = self.relu2(x)
        
# #         x = self.conv3(x)
# #         x = self.relu3(x)
# #         x = self.conv4(x)
# #         x = self.relu4(x)
        
# #         x = self.conv5(x)
# #         x = self.relu5(x)
# #         x = self.conv6(x)
# #         x = self.relu6(x)

#         x = self.flatten(x)
#         x = self.fc1(x)
#         x = self.relu3(x)
#         x = self.fc2(x)
#         return x


In [27]:
class twoDCNN(nn.Module):#input5
    def __init__(self, num_features, num_targets, hidden_size, cfg):
        self.model_name = "2dcnn"
        self.num_features = num_features
        self.num_targets = num_targets
        super(twoDCNN, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=8, kernel_size=(3, 1), bias=False)
        self.relu1 = nn.ReLU()
        self.conv2 = nn.Conv2d(in_channels=8, out_channels=16, kernel_size=(2, 1), bias=False)
        self.relu2 = nn.ReLU()
        
        # self.conv3 = nn.Conv2d(in_channels=16, out_channels=32, kernel_size=(2, 1), bias=False)
        # self.relu3 = nn.ReLU()
        # self.conv4 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=(3, 1), bias=False)
        # self.relu4 = nn.ReLU()
        
#         self.conv5 = nn.Conv2d(in_channels=64, out_channels=128, kernel_size=(2, 1), bias=False)
#         self.relu5 = nn.ReLU()
#         self.conv6 = nn.Conv2d(in_channels=128, out_channels=1024, kernel_size=(1, 1), bias=False)
#         self.relu6 = nn.ReLU()
            
        self.flatten = nn.Flatten()
        self.fc1 = nn.Linear(in_features=32, out_features=512)
        # self.fc1 = nn.Linear(in_features=80, out_features=512)
        self.relu3 = nn.ReLU()
        self.fc2 = nn.Linear(in_features=512, out_features=num_targets)
        

    def forward(self, x):
        x = self.conv1(x)
        x = self.relu1(x)
        x = self.conv2(x)
        x = self.relu2(x)
        
#         x = self.conv3(x)
#         x = self.relu3(x)
#         x = self.conv4(x)
#         x = self.relu4(x)
        
#         x = self.conv5(x)
#         x = self.relu5(x)
#         x = self.conv6(x)
#         x = self.relu6(x)

        x = self.flatten(x)
        x = self.fc1(x)
        x = self.relu3(x)
        x = self.fc2(x)
        return x

In [28]:
class StackingMLP(nn.Module):
    def __init__(self, num_features, num_targets, hidden_size, cfg):
        super().__init__()
        self.model_name = "mlp"
        cfg.num_features = num_features
        cfg.hidden_size = hidden_size
    
        self.mlp = nn.Sequential(
            nn.BatchNorm1d((num_features)),
            nn.Dropout(cfg.dropout),
            nn.utils.weight_norm(nn.Linear((cfg.num_features), cfg.hidden_size)),
            nn.PReLU(),

            
            nn.BatchNorm1d(cfg.hidden_size),
            nn.Dropout(cfg.dropout),
            nn.utils.weight_norm(nn.Linear(cfg.hidden_size, cfg.hidden_size)),
            nn.PReLU(),

                        
            nn.BatchNorm1d(cfg.hidden_size),
            nn.Dropout(cfg.dropout),
            nn.utils.weight_norm(nn.Linear(cfg.hidden_size, num_targets)),

        )
        
        self.shallow_mlp = nn.Sequential(
        
                          nn.Linear((cfg.num_features), cfg.hidden_size),
                          nn.BatchNorm1d(cfg.hidden_size),
                          nn.Dropout(cfg.dropout),
                          nn.PReLU(),
                          nn.Linear(cfg.hidden_size, cfg.hidden_size),
                          nn.BatchNorm1d(cfg.hidden_size),
                          nn.Dropout(cfg.dropout),
                          nn.PReLU(),
                          nn.Linear(cfg.hidden_size, num_targets),
                          )


    def forward(self, x):
        # if "shallow" in  stacking_exp_name:
        x = self.shallow_mlp(x)
        # else:
        #     x = self.mlp(x)
        return x

In [29]:
# sys.path.append("../../")
# from models.graphconv import GraphConv
# !python -m pip install --upgrade pip setuptools
# !pip uninstall torch -y
# !pip install torch-scatter torch-sparse torch-cluster torch-spline-conv torch-geometric
# !pip install torch -f https://pytorch-geometric.com/whl/torch-1.11.0+cu102.html

# tg.GraphConv
# a = torch.randn(3, 4).to_sparse_csr()

In [30]:
# import torch.nn.functional as F
# from torch_geometric.nn import GCNConv

# class GCN(nn.Module):
#     def __init__(self, num_features, num_targets, hidden_size, cfg):
#         self.model_name="gcn"
#         self.num_features = num_features
#         self.num_targets = num_targets
#         super().__init__()
        
#         self.gc1 = GCNConv(num_features, 16)
#         self.relu1 = nn.ReLU()
#         self.gc2= GCNConv(num_features, 16)
#         self.relu2= nn.ReLU()
#         self.gc3= GCNConv(num_features, 16)
#         self.relu3= nn.ReLU()
#         self.gc4= GCNConv(num_features, 16)
#         self.relu4= nn.ReLU()
#         self.fc1 = nn.Linear(3296, 2048)
#         self.relu5 = nn.ReLU()
#         self.fc2 = nn.Linear(2048, num_targets)
        
#         adj_mat = torch.ones(num_targets, num_targets) / num_targets
#         self.A = adj_mat.float().to(device)
#         # else:
#         #     adj_mat = (1 - torch.eye(n_classes, n_classes)) / (n_classes - 1) 
#         # self.register_buffer("A", adj_mat.float())
#     def forward(self, x):
#         x = x.unsqueeze(1)
#         x = self.relu1(self.gc1(x, self.A[None, ...]))
#         x = self.relu2(self.gc2(x))
#         x = self.relu3(self.gc3(x))
#         x = self.relu4(self.gc4(x))
#         x = self.relu5(self.fc1(x))
#         x = self.fc2(x)
#         return x

In [31]:
# ! pip install --verbose --no-cache-dir torch-scatter
# ! pip install --verbose --no-cache-dir torch-sparse
# ! pip install --verbose --no-cache-dir torch-cluster
# ! pip install --verbose --no-cache-dir torch-spline-conv (optional)
# ! pip install torch-geometric


In [32]:
import typing as tp

class MLP(nn.Module):
    """Stacked Dense layers"""
    
    def __init__(
        self, n_features_list: tp.List[int], use_tail_as_out: bool=False,
        drop_rate: float=0.0, use_bn: bool=False, use_wn: bool=False,
        activ:str="relu", block_name: str="LBAD"
    ):
        """"""
        super(MLP, self).__init__()
        n_layers = len(n_features_list) - 1
        block_class = {
            "LBAD": LBAD, "BDLA": BDLA, "LABD": LABD}[block_name]
        layers = []
        for i in range(n_layers):
            in_feats, out_feats = n_features_list[i: i + 2]
            if i == n_layers - 1 and use_tail_as_out:
                if block_name in ["BDLA"]:
                    layer = block_class(in_feats, out_feats, drop_rate, use_bn,  use_wn, "identity")
                else:
                    layer = nn.Linear(in_feats, out_feats)
                    if use_wn:
                        layer = nn.utils.weight_norm(layer)
            else:
                layer = block_class(in_feats, out_feats, drop_rate, use_bn,  use_wn, activ)
            layers.append(layer)
                
        self.layers = nn.Sequential(*layers)
        
    def forward(self, x: torch.Tensor) -> torch.Tensor:
        """"""
        return self.layers(x)


class CNNStacking1d(nn.Module):
    """1D-CNN for Stacking."""
    
    def __init__(
        self, n_models: int,
        n_channels_list: tp.List[int], use_bias: bool=False,
        kwargs_head: tp.Dict={},
    ):
        """"""
        super(CNNStacking1d, self).__init__()
        self.n_conv_layers = len(n_channels_list) - 1
        for i in range(self.n_conv_layers):
            in_ch = n_channels_list[i]
            out_ch = n_channels_list[i + 1]
            layer = nn.Sequential(
                nn.Conv1d(
                    in_ch, out_ch, kernel_size=3, stride=1, padding=0, bias=use_bias),
                # nn.BatchNorm1d(out_ch),
                nn.ReLU(inplace=True))
            setattr(self, "conv{}".format(i + 1), layer)
        
        kwargs_head["n_features_list"][0] = (n_models - 2 * self.n_conv_layers) * n_channels_list[-1]
        self.head = MLP(**kwargs_head)
    
    def forward(self, x: torch.FloatTensor) -> torch.Tensor:
        """"""
        bs = x.shape[0]
        h = x  # shape: (bs, n_classes, n_models)
        for i in range(self.n_conv_layers):
            h = getattr(self, "conv{}".format(i + 1))(h)
            
        h = torch.reshape(h, (bs, -1))
        h = self.head(h)
        return h
    
    
    
def get_activation(activ_name: str="relu"):
    """"""
    act_dict = {
        "relu": nn.ReLU(),
        "tanh": nn.Tanh(),
        "sigmoid": nn.Sigmoid(),
        "identity": nn.Identity()}
    if activ_name in act_dict:
        return act_dict[activ_name]
    elif re.match(r"^htanh\_\d{4}$", activ_name):
        bound = int(activ_name[-4:]) / 1000
        return nn.Hardtanh(-bound, bound)
    else:
        raise NotImplementedError

class LBAD(nn.Module):
    """Linear (-> BN) -> Activation (-> Dropout)"""
    
    def __init__(
        self, in_features: int, out_features: int, drop_rate: float=0.0,
        use_bn: bool=False, use_wn: bool=False, activ: str="relu"
    ):
        """"""
        super(LBAD, self).__init__()
        layers = [nn.Linear(in_features, out_features)]
        if use_wn:
            layers[0] = nn.utils.weight_norm(layers[0])
        
        if use_bn:
            layers.append(nn.BatchNorm1d(out_features))
        
        layers.append(get_activation(activ))
        
        if drop_rate > 0:
            layers.append(nn.Dropout(drop_rate))
        
        self.layers = nn.Sequential(*layers)
        
    def forward(self, x: torch.Tensor) -> torch.Tensor:
        """"""
        return self.layers(x)
    
    
class BDLA(nn.Module):
    """(BN -> Dropout ->) Linear -> Activation"""
    
    def __init__(
        self, in_features: int, out_features: int, drop_rate: float=0.0,
        use_bn: bool=False, use_wn: bool=False, activ: str="relu"
    ):
        """"""
        super(BDLA, self).__init__()
        layers = []
        if use_bn:
            layers.append(nn.BatchNorm1d(in_features))
            
        if drop_rate > 0:
            layers.append(nn.Dropout(drop_rate))
        
        layers.append(nn.Linear(in_features, out_features))
        if use_wn:
            layers[-1] = nn.utils.weight_norm(layers[-1])
            
        layers.append(get_activation(activ))
        
        self.layers = nn.Sequential(*layers)
        
    def forward(self, x: torch.Tensor) -> torch.Tensor:
        """"""
        return self.layers(x)
    

class LABD(nn.Module):
    """Linear -> Activation (-> BN -> Dropout) """
    
    def __init__(
        self, in_features: int, out_features: int, drop_rate: float=0.0,
        use_bn: bool=False, use_wn: bool=False, activ: str="relu"
    ):
        """"""
        super(LABD, self).__init__()
        layers = [nn.Linear(in_features, out_features), get_activation(activ)]
        
        if use_wn:
            layers[0] = nn.utils.weight_norm(layers[0])
        
        if use_bn:
            layers.append(nn.BatchNorm1d(out_features))
        
        if drop_rate > 0:
            layers.append(nn.Dropout(drop_rate))
        
        self.layers = nn.Sequential(*layers)
        
    def forward(self, x: torch.Tensor) -> torch.Tensor:
        """"""
        return self.layers(x)
# # for GCNs
def vector_wise_matmul(X: torch.Tensor, W: torch.Tensor) -> torch.Tensor:
    """
    See input matrixes X as bags of vectors, and multiply corresponding weight matrices by vector.
    
    Args:
        X: Input Tensor, shape: (batch_size, **n_vectors**, in_features)
        W: Weight Tensor, shape: (**n_vectors**, out_features, in_features)
    """

    X = torch.transpose(X, 0, 1)  # shape: (n_vectors, batch_size, in_features)
    W = torch.transpose(W, 1, 2)  # shape: (n_vectors, in_features, out_features)
    H = torch.matmul(X, W)        # shape: (n_vectors, batch_size, out_features)
    H = torch.transpose(H, 0, 1)  # shape: (batch_size, n_vectors, out_features)
    
    return H


def vector_wise_shared_matmul(X: torch.Tensor, W: torch.Tensor) -> torch.Tensor:
    """
    See input matrixes X as bags of vectors, and multiply **shared** weight matrices.
    
    Args:
        X: Input Tensor, shape: (batch_size, **n_vectors**, in_features)
        W: Weight Tensor, shape: (out_features, in_features)
    """
    # W = torch.transpose(W, 0, 1)  # shape: (in_features, out_features)
    # H = torch.matmul(X, W)        # shape: (batch_size, n_vectors, out_features)
    
    H = nn.functional.linear(X, W)  # shape: (batch_size, n_vectors, out_features)
    
    return H
def _calculate_fan_in_and_fan_out_for_vwl(tensor) -> tp.Tuple[int]:
    """
    Input tensor: (n_vectors, out_features, in_features) or (out_features, in_features)
    """
    dimensions = tensor.dim()
    if dimensions < 2:
        raise ValueError("Fan in and fan out can not be computed for tensor with fewer than 2 dimensions")

    fan_in = tensor.size(-1)
    fan_out = tensor.size(-2)

    return fan_in, fan_out
    

def _calculate_correct_fan_for_vwl(tensor, mode) -> int:
    """"""
    mode = mode.lower()
    valid_modes = ['fan_in', 'fan_out']
    if mode not in valid_modes:
        raise ValueError("Mode {} not supported, please use one of {}".format(mode, valid_modes))

    fan_in, fan_out = _calculate_fan_in_and_fan_out_for_vwl(tensor)
    return fan_in if mode == 'fan_in' else fan_out


def kaiming_uniform_for_vwl(tensor, a=0, mode='fan_in', nonlinearity='leaky_relu'):
    """"""
    fan = _calculate_correct_fan_for_vwl(tensor, mode)
    gain = nn.init.calculate_gain(nonlinearity, a)
    std = gain / np.sqrt(fan)
    bound = np.sqrt(3.0) * std  # Calculate uniform bounds from standard deviation
    with torch.no_grad():
        return tensor.uniform_(-bound, bound)
class VectorWiseLinear(nn.Module):
    """
    For mini batch which have several matrices,
    see as these matrixes as bags of vectors, and multiply weight matrices by vector.
    
    input    X: (batch_size, **n_vectors**, in_features)
    weight W: (**n_vector**, out_features, in_features)
    output  Y: (batch_size, **n_vectors**, out_features)

    **Note**: For simplicity, bias is not described.
    
    X and W are can be seen as below.
    X: [
            [vec_{ 1, 1}, vec_{ 1, 2}, ... vec_{ 1, n_vectors}],
            [vec_{ 2, 1}, vec_{ 2, 2}, ... vec_{ 2, n_vectors}],
                                            .
                                            .
            [vec_{bs, 1}, vec_{bs, 2}, ... vec_{bs, n_vectors}]
        ]
    W: [
            Mat_{1}, Mat_{2}, ... , Mat_{n_vectors}
        ]
    Then Y is calclauted as:
    Y: [
        [ Mat_{1} vec_{ 1, 1}, Mat_{2} vec_{ 1, 2}, ... Mat_{n_vectors} vec_{ 1, n_vectors}],
        [ Mat_{1} vec_{ 2, 1}, Mat_{2} vec_{ 2, 2}, ... Mat_{n_vectors} vec_{ 2, n_vectors}],
        .
        .
        [ Mat_{1} vec_{bs, 1}, Mat_{2} vec_{bs, 2}, ... Mat_{n_vectors} vec_{bs, n_vectors}],
    ]
    """
    
    def __init__(
        self,
        in_features: int, out_features: int, n_vectors: int,
        bias: bool=True, weight_shared: bool=True
    ) -> None:
        """Initialize."""
        super(VectorWiseLinear, self).__init__()
        self.in_features = in_features
        self.out_features = out_features
        self.n_vectors = n_vectors
        self.weight_shared = weight_shared
        
        if self.weight_shared:
            self.weight = nn.Parameter(
                torch.Tensor(self.out_features, self.in_features)).to(device)
            self.matmul_func = vector_wise_shared_matmul
        else:
            self.weight = nn.Parameter(
                torch.Tensor(self.n_vectors, self.out_features, self.in_features))
            self.matmul_func = vector_wise_matmul
            
        if bias:
            self.bias = nn.Parameter(torch.Tensor(out_features))
        else:
            self.register_parameter('bias', None)
            
        self.reset_parameters()
        
    def reset_parameters(self) -> None:
        """Initialize weight and bias."""
        kaiming_uniform_for_vwl(self.weight, a=np.sqrt(5))
        if self.bias is not None:
            fan_in, _ = _calculate_fan_in_and_fan_out_for_vwl(self.weight)
            bound = 1 / np.sqrt(fan_in)
            nn.init.uniform_(self.bias, -bound, bound)
             
    def forward(self, X: torch.Tensor) -> torch.Tensor:
        """Forward."""
        H = self.matmul_func(X, self.weight)
        if self.bias is not None:
            H = H + self.bias
        
        return H
class GraphConv(nn.Module):
    """Basic Graph Convolution Layer."""
    
    def __init__(
        self, 
        in_channels: int, out_channels: int, n_nodes: int, shrare_msg: bool=True,
        model_self: bool=True, share_model_self: bool=True,
        bias: bool=True, share_bias: bool=True
    ) -> None:
        """Intialize."""
        self.in_channels = in_channels
        self.out_channels = out_channels
        self.n_nodes = n_nodes
        self.model_self = model_self
        super(GraphConv, self).__init__()
        
        # # message
        self.msg = VectorWiseLinear(
            in_channels, out_channels, n_nodes, False, shrare_msg)

        # # self-modeling
        if model_self:
            self.model_self = VectorWiseLinear(
                in_channels, out_channels, n_nodes, False, share_model_self)
        
        # # bias
        if bias:
            if share_bias:
                self.bias = nn.Parameter(torch.Tensor(out_channels))
            else:
                self.bias = nn.Parameter(torch.Tensor(n_nodes, out_channels))
            bound = 1 / np.sqrt(out_channels)
            nn.init.uniform_(self.bias, -bound, bound)
        else:
            self.register_parameter('bias', None)
            
    
    def forward(self, X: torch.Tensor, A: torch.Tensor, W: torch.Tensor=None) -> torch.Tensor:
        """Forward.
        
        Args:
            X: (batch_size, n_nodes, n_channels)
                Array which represents bags of vectors.
                X[:, i, :] are corresponded to feature vectors of node i.
            A: (batch_size, n_nodes, n_nodes)
                Array which represents adjacency matrices.
                A[:, i, j] are corresponded to weights (scalar) of edges from node j to node i.
            W: (batch_size, n_nodes, n_nodes)
                Array which represents weight matrices between nodes.
        """
        if W is not None:
            A = A * W  # shape: (batch_size, n_nodes, n_nodes)
        
        # # update message
        M = X  #  shape: (batch_size, n_nodes, in_channels)
        # # # send message
        M = self.msg(M)  # shape: (batch_size, n_nodes, out_channels)
        # # # aggregate
        M = torch.matmul(A, M)  # shape: (batch_size, n_nodes, out_channels)
            
        # # update node
        # # # self-modeling
        H = M
        if self.model_self:
            H = H + self.model_self(X)
        if self.bias is not None:
            H = H + self.bias
        
        return H

In [33]:
class GCN(nn.Module):
    """GCN for Stacking."""
    
    # def __init__(
    #     self, n_classes: int,
    #     n_channels_list: tp.List[int],
    #     add_self_loop: bool=False,
    #     kwargs_head: tp.Dict={},
    # ):
    def __init__(self, num_features, num_targets, hidden_size, cfg):
        self.model_name = "GCN"
        n_classes = 1
        n_channels_list =  [5, 16, 16, 16, 16, 16,16]#[8, 16, 16, 16, 16, 16, 16,]
        add_self_loop = True
        kwargs_head = {
            "n_features_list": [-1, 768, 1],# [-1, 2048, 1]
            "use_tail_as_out": True,
            "drop_rate": 0.8,
            "use_bn": False,
            "use_wn": True,
            "block_name": "LABD",
        }
        
        
        super().__init__()
        self.n_conv_layers = len(n_channels_list) - 1
        for i in range(self.n_conv_layers):
            in_ch = n_channels_list[i]
            out_ch = n_channels_list[i + 1]
            # layer = CustomGraphConv(in_ch, out_ch, n_classes)
            layer = GraphConv(
                in_ch, out_ch, n_classes,
                shrare_msg=False, share_model_self=False, share_bias=False)
            setattr(self, "conv{}".format(i + 1), layer)
        
        self.relu = nn.ReLU(inplace=True)
        if add_self_loop:
            adj_mat = torch.ones(n_classes, n_classes) / n_classes
        else:
            adj_mat = (1 - torch.eye(n_classes, n_classes)) / (n_classes - 1) 
        self.register_buffer("A", adj_mat.float())
               
        kwargs_head["n_features_list"][0] = n_classes * n_channels_list[-1]
        self.head = MLP(**kwargs_head)
    
    def forward(self, X: torch.FloatTensor) -> torch.Tensor:
        """"""
        X = X.unsqueeze(1)
        bs, n_classes = X.shape[:2]
        H = X  # shape: (bs, n_classes, n_models)
        for i in range(self.n_conv_layers):
            H = getattr(self, "conv{}".format(i + 1))(H, self.A[None, ...])
            H = self.relu(H)
        
        h = torch.reshape(H, (bs, -1))
        h = self.head(h)
        return h

# TabNet

In [34]:
!pip install pytorch-tabnet==

ERROR: Could not find a version that satisfies the requirement pytorch-tabnet== (from versions: 0.1.0, 0.1.1, 0.1.2, 1.0.0, 1.0.1, 1.0.2, 1.0.3, 1.0.4, 1.0.5, 1.0.6, 1.1.0, 1.2.0, 2.0.0, 2.0.1, 3.0.0, 3.1.0, 3.1.1)
ERROR: No matching distribution found for pytorch-tabnet==


In [35]:
from pytorch_tabnet.tab_model import TabNetClassifier, TabNetRegressor
from pytorch_tabnet.pretraining import TabNetPretrainer
# from pytorch_tabnet.augmentations import RegressionSMOTE

def train_tabnet(X_train, y_train, X_valid, y_valid, MODEL, save_dir, fold_num, seed, max_epochs=200):
    print("start unsupervised fitting")
    unsupervised_model = TabNetPretrainer(
        optimizer_fn=torch.optim.Adam,
        optimizer_params=dict(lr=2e-2),
        mask_type='entmax', # "sparsemax",
        verbose=10,
        device_name="cuda"
        # n_shared_decoder=1, # nb shared glu for decoding
        # n_indep_decoder=1, # nb independent glu for decoding
    )
    unsupervised_model.fit(
        X_train=X_train,
        eval_set=[X_valid],
        max_epochs=max_epochs , patience=5,
        batch_size=2048, virtual_batch_size=128,
        num_workers=0,
        drop_last=False,
        pretraining_ratio=0.8,
    )
    unsupervised_model_path =  f"{save_dir}unsupervised_fold{fold_num}_seed{seed}"
    unsupervised_model.save_model(unsupervised_model_path)
    # loaded_pretrain = TabNetPretrainer()
    # loaded_pretrain.load_model('./test_pretrain.zip')
    
    model = MODEL(optimizer_fn=torch.optim.Adam,
                       optimizer_params=dict(lr=2e-2),
                       scheduler_params={"step_size":10, # how to use learning rate scheduler
                                         "gamma":0.9,
                                        },
                     scheduler_fn=torch.optim.lr_scheduler.StepLR,
                    
                  # scheduler_params=dict(mode="min",
                  #                          patience=5,
                  #                          min_lr=1e-5,
                  #                          factor=0.9,),
                  #   scheduler_fn=torch.optim.lr_scheduler.ReduceLROnPlateau,

                    verbose=10,
                  device_name="cuda"

                      )
    print("end unsupervised fitting")
    print("start model fitting")
    # aug = RegressionSMOTE(p=0.2)

    model.fit(
        X_train=X_train, y_train=y_train,
        eval_set=[(X_train, y_train), (X_valid, y_valid)],
        eval_name=['train', 'valid'],
        eval_metric=['rmsle', 'mae', 'rmse', 'mse'],
        max_epochs=max_epochs,
        patience=50,
        batch_size=1024, virtual_batch_size=128,
        num_workers=0,
        drop_last=False,
        # augmentations=aug, #aug
        from_unsupervised=unsupervised_model,
    )
    model_path =  f"{save_dir}tabnet_fold{fold_num}_seed{seed}"
    LOGGER.info(model.save_model(model_path))
    
    print("end model fitting")
    return model
 

In [36]:
class oneDCNNCFG:
    max_grad_norm=1000
    gradient_accumulation_steps=1
    hidden_size=256
    dropout=0.3
    lr=1e-4
    batch_size=128
    epochs=50
    weight_decay=1e-5
    n_fold = 5
    
    features = oof_dfs.columns[:-1]
   

    
train =  pd.read_csv(INPUT_DIR+'train.csv')
# train['score_map'] = train['score'].map({0.00: 0, 0.25: 1, 0.50: 2, 0.75: 3, 1.00: 4})
# Fold = StratifiedKFold(n_splits=oneDCNNCFG.n_fold, shuffle=True, random_state=CFG.seed)
# for n, (train_index, val_index) in enumerate(Fold.split(train, train['score_map'])):
#     train.loc[val_index, 'fold'] = int(n)
# train['fold'] = train['fold'].astype(int)
# folds = train
# display(train.groupby('fold').size())

# train = train.merge(oof_dfs ,on="id")#
# target = train["score"]
# train = train[oof_dfs.columns[:-1]]


In [37]:
def create_folds(data, num_splits):
    # we create a new column called kfold and fill it with -1
    data["fold"] = -1
    
    # the next step is to randomize the rows of the data
    data = data.sample(frac=1).reset_index(drop=True)

    # calculate number of bins by Sturge's rule
    # I take the floor of the value, you can also
    # just round it
    num_bins = int(np.floor(1 + np.log2(len(data))))
    
    # bin targets
    data.loc[:, "bins"] = pd.cut(
        data["score"], bins=num_bins, labels=False
    )
    
    # initiate the kfold class from model_selection module
    kf = model_selection.StratifiedKFold(n_splits=num_splits, shuffle=True, random_state=42)
    # kf = model_selection.StratifiedGroupKFold(n_splits=num_splits, shuffle=True, random_state=42)

    # fill the new kfold column
    # note that, instead of targets, we use bins!
    for f, (t_, v_) in enumerate(kf.split(X=data, y=data.bins.values)):
        data.loc[v_, 'fold'] = f
    
    # drop the bins column
    data = data.drop("bins", axis=1)

    # return dataframe with folds
    return data


folds = create_folds(train, oneDCNNCFG.n_fold)
target = train["score"]
train = train.merge(oof_dfs, on="id")[oof_dfs.columns[:-1]]
# train
# display(oof_dfs.head())
# display(train.head())

In [38]:
from tqdm.notebook import tqdm

In [39]:
def train_stack(train_loader, model, optimizer, epoch, scheduler, device,CFG):
    
    losses = AverageMeter()

    model.train()

    for step, (x, y) in enumerate(train_loader):
        x, y = x.to(device, non_blocking=True), y.to(device, non_blocking=True)
        batch_size = x.size(0)
        pred = model(x)
        loss = nn.BCEWithLogitsLoss()(pred, y.unsqueeze(1))
        losses.update(loss.item(), batch_size)
        if CFG.gradient_accumulation_steps > 1:
            loss = loss / CFG.gradient_accumulation_steps

        loss.backward()

        grad_norm = torch.nn.utils.clip_grad_norm_(model.parameters(), CFG.max_grad_norm)

        if (step + 1) % CFG.gradient_accumulation_steps == 0:
            scheduler.step()
            optimizer.step()
            optimizer.zero_grad()
        
    return losses.avg


def valid_stack(valid_loader, model, device, CFG):
    
    losses = AverageMeter()

    model.eval()
    val_preds = []
    y_true =[]
    for step, (x, y) in enumerate(valid_loader):
        
        x, y = x.to(device, non_blocking=True), y.to(device, non_blocking=True)
        batch_size = x.size(0)

        with torch.no_grad():
            pred = model(x)
            
        loss = nn.BCEWithLogitsLoss()(pred, y.unsqueeze(1))
        losses.update(loss.item(), batch_size)

        # print(sum(pred))
        val_preds.append(pred.sigmoid().detach().cpu().numpy())
        y_true.append(y.detach().cpu().numpy())
#         val_preds.append(pred.tanh().detach().cpu().numpy())

        if CFG.gradient_accumulation_steps> 1:
            loss = loss / CFG.gradient_accumulation_steps
    val_preds = np.concatenate(val_preds)
    y_true= np.concatenate(y_true)
    try:
        score = get_score(y_true, val_preds.reshape(-1,))
    except:
        score = None
    return losses.avg, val_preds, score



class AverageMeter(object):
    """Computes and stores the average and current value"""
    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count


In [40]:
def run_single_nn(cfg, train, folds, features, target, device, fold_num, seed, MODEL,save_dir,):
    seed_everything(seed=seed)
    trn_idx = folds[folds["fold"]!=fold_num].index
    val_idx = folds[folds["fold"]==fold_num].index
    
    train_target = target[trn_idx]
    valid_target = target[val_idx]
    
    train_folds = train.loc[trn_idx].reset_index(drop=True)
    valid_folds = train.loc[val_idx].reset_index(drop=True)
    
    # s = QuantileTransformer(n_quantiles=50, random_state=seed, output_distribution="normal")
    # train_folds[features] = s.fit_transform(train_folds[features])
    # valid_folds[features] = s.transform(valid_folds[features])
    if cfg.like_sklearn:
       
        model = train_tabnet(train_folds.values, train_target.values.reshape(-1,1), valid_folds.values, valid_target.values.reshape(-1,1), MODEL, save_dir, fold_num, seed,)
        tra_pred = model.predict(train_folds.values)
        val_pred = model.predict(valid_folds.values)
        oof = np.zeros((len(train), 1))
        oof[val_idx] = val_pred
        
        tra_score = get_score(train_target.values, tra_pred)
        val_score = get_score(valid_target.values, val_pred)
        LOGGER.info(f"train[{tra_score}], val[{val_score}]")
        # save tabnet model
        
        # # define new model with basic parameters and load state dict weights
        # loaded_clf = TabNetClassifier()
        # loaded_clf.load_model(saved_filepath)
    else:
        model = MODEL(
                num_features=len(cfg.features), num_targets=1, hidden_size=cfg.hidden_size, cfg=cfg
        )
        model.to(device)#, non_blocking=True)

        train_dataset = StackingDataset(train_folds, features, train_target, model_name=model.model_name)
        valid_dataset = StackingDataset(valid_folds, features, valid_target, model_name=model.model_name)

        train_loader = DataLoader(train_dataset, batch_size=cfg.batch_size, shuffle=True, 
                                  num_workers=4, pin_memory=False, drop_last=True)
        valid_loader = DataLoader(valid_dataset, batch_size=cfg.batch_size, shuffle=False, 
                                  num_workers=4, pin_memory=False, drop_last=False)


        optimizer = optim.Adam(model.parameters(), lr=cfg.lr, weight_decay=cfg.weight_decay)
        scheduler = optim.lr_scheduler.OneCycleLR(optimizer=optimizer, pct_start=0.1, div_factor=1e3, 
                                                  max_lr=1e-2, epochs=cfg.epochs, steps_per_epoch=len(train_loader))

        # train & validate
        best_score = -np.inf
        improve_count = 0
        for epoch in range(cfg.epochs):
            train_loss = train_stack(train_loader, model, optimizer, epoch, scheduler, device, cfg)
            valid_loss, val_pred, score = valid_stack(valid_loader, model, device, cfg)
            if score is None:
                break
            if score>best_score:
                LOGGER.info(f"epoch [{epoch}] update {best_score} to {score}")
                best_score = score
                oof = np.zeros((len(train), 1))
                oof[val_idx] = val_pred
                torch.save(model.state_dict(),f"{save_dir}{model.model_name}_fold{fold_num}_seed{seed}.pth")
                improve_count=0
            else:
                print(f"not improve {score}")
                improve_count+=1

            if improve_count>10:
                break
    return oof
def run_kfold_nn(cfg, train, folds, features, target, device, n_fold, seed,MODEL, save_dir,):
    print("run kfold nn")
    oof = np.zeros((len(train), 1))
    for fold_num in range(n_fold):
        LOGGER.info(f"fold {fold_num}")
        _oof = run_single_nn(cfg, train, folds, features, target, device, fold_num, seed, MODEL,save_dir,)
        oof += _oof
    score = get_score(target.values, oof)
    LOGGER.info("="*10)
    LOGGER.info(f"oof score {score}")
    return oof

In [41]:
like_sklearn = False
if "1dcnn" in stacking_exp_name:
    stacking_model = oneDCNN
elif "2dcnn" in stacking_exp_name:
    stacking_model = twoDCNN
elif "mlp" in stacking_exp_name:
    stacking_model = StackingMLP
elif "gcn" in stacking_exp_name:# 
    device = torch.device("cpu")
    stacking_model = GCN
elif "tabnet" in stacking_exp_name:
    stacking_model = TabNetRegressor
    like_sklearn = True#学習をmodel.fitで行う
    
oneDCNNCFG.like_sklearn = like_sklearn

In [ ]:
oof = np.zeros((len(train), 1))
SEED = [42, 1999, 2022]

for i, seed in enumerate(SEED):
    LOGGER.info(f"set seed {seed}")
    _oof = run_kfold_nn(oneDCNNCFG, train, folds, oneDCNNCFG.features, target, device,
                        n_fold=oneDCNNCFG.n_fold, seed=seed,MODEL=stacking_model, save_dir=OUTPUT_DIR,
                       )
    oof += _oof/len(SEED)

set seed 42
fold 0


run kfold nn


In [ ]:
train["score"] = oof

train.to_csv(f"{OUTPUT_DIR}oof_df.csv",index=False)

In [ ]:
get_score(target.values, oof)

In [ ]:
# array([0.8718436039273738], dtype=object)